Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST_sanitized.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (8971, 28, 28) (8971,)
Test set (8676, 28, 28) (8676,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (8971, 784) (8971, 10)
Test set (8676, 784) (8676, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### logistic model

In [12]:
batch_size = 128

l2_param = 0.001

graph = tf.Graph()
with graph.as_default():
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) \
    + l2_param * (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)



num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 22.166344
Minibatch accuracy: 10.2%
Validation accuracy: 10.4%
Minibatch loss at step 500: 2.724553
Minibatch accuracy: 75.0%
Validation accuracy: 72.9%
Minibatch loss at step 1000: 1.691484
Minibatch accuracy: 77.3%
Validation accuracy: 75.5%
Minibatch loss at step 1500: 1.329941
Minibatch accuracy: 79.7%
Validation accuracy: 77.0%
Minibatch loss at step 2000: 0.895445
Minibatch accuracy: 82.8%
Validation accuracy: 78.6%
Minibatch loss at step 2500: 0.758590
Minibatch accuracy: 86.7%
Validation accuracy: 79.3%
Minibatch loss at step 3000: 0.892571
Minibatch accuracy: 79.7%
Validation accuracy: 80.0%
Minibatch loss at step 3500: 0.803715
Minibatch accuracy: 82.0%
Validation accuracy: 79.7%
Minibatch loss at step 4000: 0.662683
Minibatch accuracy: 85.2%
Validation accuracy: 79.5%
Minibatch loss at step 4500: 0.871705
Minibatch accuracy: 77.3%
Validation accuracy: 78.8%
Minibatch loss at step 5000: 0.590666
Minibatch accuracy: 84.4%
Validation accura

### with hidden layer

In [14]:
hidden_nodes = 1024
batch_size = 128

l2_alpha = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weightsA = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biasesA = tf.Variable(tf.zeros([hidden_nodes]))
  
  weightsB = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biasesB = tf.Variable(tf.zeros([num_labels]))

  
  # Training computation.
  logitsA = tf.matmul(tf_train_dataset, weightsA) + biasesA
  reuled = tf.nn.relu(logitsA)
  logitsB = tf.matmul(reuled, weightsB) + biasesB
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logitsB, tf_train_labels)) \
    + l2_alpha * (tf.nn.l2_loss(weightsA) + tf.nn.l2_loss(weightsB) \
                  + tf.nn.l2_loss(biasesA) + tf.nn.l2_loss(biasesB))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logitsB)
  valid_prediction = tf.nn.softmax(
     tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weightsA) + biasesA), weightsB) + biasesB)
  test_prediction = tf.nn.softmax(
     tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weightsA) + biasesA), weightsB) + biasesB)


num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("------")
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 629.755737
Minibatch accuracy: 14.1%
Validation accuracy: 28.6%
------
Minibatch loss at step 1000: 118.754608
Minibatch accuracy: 78.9%
Validation accuracy: 76.9%
------
Minibatch loss at step 2000: 41.753010
Minibatch accuracy: 85.9%
Validation accuracy: 82.7%
------
Minibatch loss at step 3000: 15.628781
Minibatch accuracy: 86.7%
Validation accuracy: 85.2%
------
Minibatch loss at step 4000: 6.003121
Minibatch accuracy: 89.1%
Validation accuracy: 85.9%
------
Minibatch loss at step 5000: 2.416358
Minibatch accuracy: 92.2%
Validation accuracy: 86.6%
------
Test accuracy: 93.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

### logistic model

In [15]:
batch_size = 128

l2_param = 0.001

graph = tf.Graph()
with graph.as_default():
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) \
    + l2_param * (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)



num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    
    offset = (step * batch_size) % (10*batch_size - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.154793
Minibatch accuracy: 10.9%
Validation accuracy: 12.6%
Minibatch loss at step 500: 1.977331
Minibatch accuracy: 93.0%
Validation accuracy: 68.5%
Minibatch loss at step 1000: 1.167139
Minibatch accuracy: 99.2%
Validation accuracy: 70.1%
Minibatch loss at step 1500: 0.783426
Minibatch accuracy: 99.2%
Validation accuracy: 71.7%
Minibatch loss at step 2000: 0.535509
Minibatch accuracy: 100.0%
Validation accuracy: 72.4%
Minibatch loss at step 2500: 0.412128
Minibatch accuracy: 99.2%
Validation accuracy: 73.3%
Minibatch loss at step 3000: 0.283262
Minibatch accuracy: 100.0%
Validation accuracy: 73.7%
Minibatch loss at step 3500: 0.252746
Minibatch accuracy: 99.2%
Validation accuracy: 74.2%
Minibatch loss at step 4000: 0.206625
Minibatch accuracy: 100.0%
Validation accuracy: 74.5%
Minibatch loss at step 4500: 0.202559
Minibatch accuracy: 100.0%
Validation accuracy: 74.8%
Minibatch loss at step 5000: 0.189557
Minibatch accuracy: 100.0%
Validation a

### network

In [22]:
hidden_nodes = 1024
batch_size = 128

l2_alpha = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weightsA = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biasesA = tf.Variable(tf.zeros([hidden_nodes]))
  
  weightsB = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biasesB = tf.Variable(tf.zeros([num_labels]))

  
  # Training computation.
  logitsA = tf.matmul(tf_train_dataset, weightsA) + biasesA
  reuled = tf.nn.relu(logitsA)
  logitsB = tf.matmul(reuled, weightsB) + biasesB
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logitsB, tf_train_labels)) \
    + l2_alpha * (tf.nn.l2_loss(weightsA) + tf.nn.l2_loss(weightsB) \
                  + tf.nn.l2_loss(biasesA) + tf.nn.l2_loss(biasesB))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logitsB)
  valid_prediction = tf.nn.softmax(
     tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weightsA) + biasesA), weightsB) + biasesB)
  test_prediction = tf.nn.softmax(
     tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weightsA) + biasesA), weightsB) + biasesB)


num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    
    offset = (step * batch_size) % (100 * batch_size - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("------")
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 615.190308
Minibatch accuracy: 17.2%
Validation accuracy: 37.0%
------
Minibatch loss at step 1000: 115.009415
Minibatch accuracy: 96.9%
Validation accuracy: 79.3%
------
Minibatch loss at step 2000: 42.230274
Minibatch accuracy: 100.0%
Validation accuracy: 81.2%
------
Minibatch loss at step 3000: 15.558963
Minibatch accuracy: 100.0%
Validation accuracy: 82.2%
------
Minibatch loss at step 4000: 5.826416
Minibatch accuracy: 99.2%
Validation accuracy: 83.1%
------
Minibatch loss at step 5000: 2.277312
Minibatch accuracy: 100.0%
Validation accuracy: 83.8%
------
Minibatch loss at step 6000: 0.985941
Minibatch accuracy: 98.4%
Validation accuracy: 83.2%
------
Minibatch loss at step 7000: 0.496592
Minibatch accuracy: 100.0%
Validation accuracy: 84.0%
------
Minibatch loss at step 8000: 0.339747
Minibatch accuracy: 99.2%
Validation accuracy: 84.0%
------
Minibatch loss at step 9000: 0.255071
Minibatch accuracy: 100.0%
Validation accuracy: 84.1%
------


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [21]:
hidden_nodes = 1024
batch_size = 128

l2_alpha = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weightsA = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biasesA = tf.Variable(tf.zeros([hidden_nodes]))
  
  weightsB = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biasesB = tf.Variable(tf.zeros([num_labels]))

  
  # Training computation.
  logitsA = tf.matmul(tf_train_dataset, weightsA) + biasesA
  dropouted = tf.nn.dropout(logitsA, 0.5)
  reuled = tf.nn.relu(dropouted)
  logitsB = tf.matmul(reuled, weightsB) + biasesB
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logitsB, tf_train_labels)) \
    + l2_alpha * (tf.nn.l2_loss(weightsA) + tf.nn.l2_loss(weightsB) \
                  + tf.nn.l2_loss(biasesA) + tf.nn.l2_loss(biasesB))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weightsA) + biasesA), weightsB) + biasesB)
  valid_prediction = tf.nn.softmax(
     tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weightsA) + biasesA), weightsB) + biasesB)
  test_prediction = tf.nn.softmax(
     tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weightsA) + biasesA), weightsB) + biasesB)


num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("------")
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 737.704041
Minibatch accuracy: 14.8%
Validation accuracy: 33.2%
------
Minibatch loss at step 1000: 120.503410
Minibatch accuracy: 67.2%
Validation accuracy: 78.3%
------
Minibatch loss at step 2000: 42.308918
Minibatch accuracy: 82.8%
Validation accuracy: 81.0%
------
Minibatch loss at step 3000: 16.083473
Minibatch accuracy: 83.6%
Validation accuracy: 83.4%
------
Minibatch loss at step 4000: 6.181463
Minibatch accuracy: 85.9%
Validation accuracy: 84.8%
------
Minibatch loss at step 5000: 2.554088
Minibatch accuracy: 88.3%
Validation accuracy: 85.7%
------
Minibatch loss at step 6000: 1.548118
Minibatch accuracy: 78.9%
Validation accuracy: 86.1%
------
Minibatch loss at step 7000: 0.961591
Minibatch accuracy: 84.4%
Validation accuracy: 85.3%
------
Minibatch loss at step 8000: 0.741113
Minibatch accuracy: 82.8%
Validation accuracy: 85.6%
------
Minibatch loss at step 9000: 0.687137
Minibatch accuracy: 85.9%
Validation accuracy: 86.0%
------
Minib

### overfitting

In [23]:
hidden_nodes = 1024
batch_size = 128

l2_alpha = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weightsA = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biasesA = tf.Variable(tf.zeros([hidden_nodes]))
  
  weightsB = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biasesB = tf.Variable(tf.zeros([num_labels]))

  
  # Training computation.
  logitsA = tf.matmul(tf_train_dataset, weightsA) + biasesA
  dropouted = tf.nn.dropout(logitsA, 0.5)
  reuled = tf.nn.relu(dropouted)
  logitsB = tf.matmul(reuled, weightsB) + biasesB
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logitsB, tf_train_labels)) \
    + l2_alpha * (tf.nn.l2_loss(weightsA) + tf.nn.l2_loss(weightsB) \
                  + tf.nn.l2_loss(biasesA) + tf.nn.l2_loss(biasesB))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weightsA) + biasesA), weightsB) + biasesB)
  valid_prediction = tf.nn.softmax(
     tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weightsA) + biasesA), weightsB) + biasesB)
  test_prediction = tf.nn.softmax(
     tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weightsA) + biasesA), weightsB) + biasesB)


num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    
    offset = (step * batch_size) % (100 * batch_size - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("------")
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 797.730469
Minibatch accuracy: 10.9%
Validation accuracy: 29.5%
------
Minibatch loss at step 1000: 119.816566
Minibatch accuracy: 86.7%
Validation accuracy: 79.1%
------
Minibatch loss at step 2000: 43.698536
Minibatch accuracy: 88.3%
Validation accuracy: 81.6%
------
Minibatch loss at step 3000: 16.219973
Minibatch accuracy: 95.3%
Validation accuracy: 82.4%
------
Minibatch loss at step 4000: 6.130980
Minibatch accuracy: 96.1%
Validation accuracy: 83.5%
------
Minibatch loss at step 5000: 2.446909
Minibatch accuracy: 96.9%
Validation accuracy: 84.0%
------
Minibatch loss at step 6000: 1.134776
Minibatch accuracy: 96.9%
Validation accuracy: 83.5%
------
Minibatch loss at step 7000: 0.679845
Minibatch accuracy: 96.1%
Validation accuracy: 84.1%
------
Minibatch loss at step 8000: 0.512542
Minibatch accuracy: 94.5%
Validation accuracy: 83.8%
------
Minibatch loss at step 9000: 0.401787
Minibatch accuracy: 96.1%
Validation accuracy: 84.0%
------
Minib

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [15]:
layers = [image_size * image_size, 1024, 10]
dropout = 0.5
batch_size = 128
learning_rate = 0.6
l2_alpha = 0.001
num_steps = 6001

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables that is weights
    weights = []
    biases = []
    for layer_idx in range(len(layers)-1):
        layer_input = layers[layer_idx]
        layer_output = layers[layer_idx+1]

        layer_weights = tf.Variable(tf.truncated_normal([layer_input, layer_output]))
        layer_biases = tf.Variable(tf.zeros([layer_output]))

        weights.append(layer_weights)
        biases.append(layer_biases)

    
    # Training computation.
    x = tf_train_dataset
    for w, b in zip(weights, biases)[:-1]:
        x = tf.matmul(x, w) + b
        x = tf.nn.dropout(x, dropout)
        x = tf.nn.relu(x)
    x = tf.matmul(x, weights[-1]) + biases[-1]   

    # Loss
    loss_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(x, tf_train_labels)
    loss_penalty = l2_alpha * (sum(map(tf.nn.l2_loss, weights)) + sum(map(tf.nn.l2_loss, biases)))
    loss = tf.reduce_mean(loss_cross_entropy + loss_penalty)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Activation
    activate_layer = lambda layer_input, (w, b): tf.nn.relu(tf.matmul(layer_input, w) + b)
    predict = lambda network_input: tf.nn.softmax(reduce(activate_layer, zip(weights, biases), network_input))

    # Predictions for the training, validation, and test data.
    train_prediction = predict(tf_train_dataset)
    valid_prediction = predict(tf_valid_dataset)
    test_prediction = predict(tf_test_dataset)

                               
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 200 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("------")
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 741.213989
Minibatch accuracy: 12.5%
Validation accuracy: 38.3%
------
Minibatch loss at step 200: 284.125427
Minibatch accuracy: 75.0%
Validation accuracy: 76.9%
------
Minibatch loss at step 400: 211.155884
Minibatch accuracy: 78.9%
Validation accuracy: 77.9%
------
Minibatch loss at step 600: 165.988724
Minibatch accuracy: 78.9%
Validation accuracy: 77.0%
------
Minibatch loss at step 800: 126.605431
Minibatch accuracy: 75.8%
Validation accuracy: 76.9%
------
Minibatch loss at step 1000: 102.910568
Minibatch accuracy: 78.1%
Validation accuracy: 78.3%
------
Minibatch loss at step 1200: 78.312386
Minibatch accuracy: 78.9%
Validation accuracy: 78.7%
------
Minibatch loss at step 1400: 60.804146
Minibatch accuracy: 75.8%
Validation accuracy: 79.9%
------
Minibatch loss at step 1600: 47.294006
Minibatch accuracy: 85.9%
Validation accuracy: 80.0%
------
Minibatch loss at step 1800: 36.885132
Minibatch accuracy: 85.9%
Validation accuracy: 81.5%
------